In [1]:
from pyspark.sql import DataFrame

# Define the cleanList method
def cleanList(self, column_name):
    # Implement the logic to clean the list in the specified column
    # For example, let's assume we want to remove any null values from the list
    return self.withColumn(column_name, self[column_name].isNull())

# Create a class A
class A:
    def __init__(self, dataframe):
        # Extend the DataFrame class with the cleanList method
        DataFrame.cleanList = cleanList

        self.dataframe = dataframe

    def clean_list(self, column_name):
        self.dataframe = self.dataframe.cleanList(column_name)
        return self.dataframe

# Test the cleanList method
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Create a sample DataFrame
data = [("Alice", [1, None, 3]), ("Bob", [4, 5, 6]), ("Charlie", [None, 8, 9])]
df = spark.createDataFrame(data, ["name", "numbers"])

# Create an instance of class A
a = A(df)

# Call the cleanList method from the constructor
cleaned_df = a.clean_list("numbers")

# Show the cleaned DataFrame
cleaned_df.show()

23/12/04 15:45:02 WARN Utils: Your hostname, tntuan0910 resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface wlp0s20f3)
23/12/04 15:45:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/04 15:45:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/04 15:45:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+-------+-------+
|   name|numbers|
+-------+-------+
|  Alice|  false|
|    Bob|  false|
|Charlie|  false|
+-------+-------+



In [2]:
# - "9047:9047"   # Web UI (HTTP)
# - "31010:31010" # ODBC/JDBC clients
# - "32010:32010" # Apache Arrow Flight clients
import json
import requests
import time
import pandas as pd

username = 'dremio'
password = 'dremio123'
headers = {'content-type': 'application/json'}
host = 'localhost'
dremioServer = f'http://{host}:9047'

def apiGet(endpoint):
    return json.loads(requests.get('{server}/api/v3/{endpoint}'
                                   .format(server=dremioServer, endpoint=endpoint), headers=headers).text)

def apiPost(endpoint, body=None):
    text = requests.post('{server}/api/v3/{endpoint}'
                        .format(server=dremioServer, endpoint=endpoint), headers=headers, data=json.dumps(body)).text

    # a post may return no data
    if text:
        return json.loads(text)
    else:
        return None

def apiPut(endpoint, body=None):
    return requests.put('{server}/api/v3/{endpoint}'.format(server=dremioServer, endpoint=endpoint), headers=headers, data=json.dumps(body)).text

def apiDelete(endpoint):
    return requests.delete('{server}/api/v3/{endpoint}'.format(server=dremioServer, endpoint=endpoint), headers=headers)

def login(username, password):
    # we login using the old api for now
    loginData = {'userName': username, 'password': password}
    headers = {'content-type': 'application/json'}
    response = requests.post(f'{dremioServer}/apiv2/login', headers=headers, data=json.dumps(loginData))
    data = json.loads(response.text)

    # retrieve the login token
    token = data['token']
    return {'content-type': 'application/json', 'authorization': '_dremio{authToken}'.format(authToken=token)}

def querySQL(query):
  queryResponse = apiPost('sql', body={'sql': query})
  jobid = queryResponse['id']
  # result = apiGet('job/{}/results'.format(jobid)) 
  return jobid

headers = login(username, password)

In [5]:
location = 'home.albums'
query = f"SELECT count(*) FROM {location}"
jobid = querySQL(query)
print('Waiting for the job to complete...')
job_status = requests.request("GET", dremioServer + "/api/v3/job/" +jobid, headers=headers).json()['jobState']

while job_status != 'COMPLETED':
    time.sleep(1)
    print(job_status)
    job_status = requests.request("GET", dremioServer + "/api/v3/job/" +jobid, headers=headers).json()['jobState']

response = requests.request("GET", dremioServer + f"/api/v3/job/"+jobid+"/results", headers=headers)
df = pd.DataFrame(response.json()['rows'])
df



Waiting for the job to complete...
PLANNING
RUNNING


,EXPR$0
0,9375


In [8]:
from sqlalchemy import create_engine
username = 'dremio'
password = 'dremio123'
headers = {'content-type': 'application/json'}
host = 'localhost'
port = 32010
path = f"dremio+flight://{username}:{password}@{host}:{port}/home"
engine = create_engine(path)
result = engine.execute('SELECT * FROM albums').fetchall()
print(result)

/tmp/ipykernel_9927/3045977396.py:8: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'sqlalchemy_dremio.flight.DremioDialect_flight'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine(path)


AttributeError: 'Engine' object has no attribute 'execute'

E1206 22:39:16.039064152   11548 ssl_transport_security.cc:1501]       Handshake failed with fatal error SSL_ERROR_SSL: error:0A00010B:SSL routines::wrong version number.
E1206 22:39:51.038952017   11548 ssl_transport_security.cc:1501]       Handshake failed with fatal error SSL_ERROR_SSL: error:0A00010B:SSL routines::wrong version number.
E1206 22:39:56.039275963   11548 ssl_transport_security.cc:1501]       Handshake failed with fatal error SSL_ERROR_SSL: error:0A00010B:SSL routines::wrong version number.
E1206 22:40:26.039051170   11548 ssl_transport_security.cc:1501]       Handshake failed with fatal error SSL_ERROR_SSL: error:0A00010B:SSL routines::wrong version number.
E1206 22:40:31.039195498   11548 ssl_transport_security.cc:1501]       Handshake failed with fatal error SSL_ERROR_SSL: error:0A00010B:SSL routines::wrong version number.
E1206 22:41:46.038647787   11548 ssl_transport_security.cc:1501]       Handshake failed with fatal error SSL_ERROR_SSL: error:0A00010B:SSL routin